We import `netgen.gui` to open the netgen GUI for displaying the ferromagnet. 
We then define the geometry, and draw the box. 
The refinement of the mesh is controlled by `H_MAX` in `ngmesh`.

In [1]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import grad, Grad  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions as magfunc
import Elastic_Functions as elfunc
import numpy as np
import scipy.sparse as sp
T_MAX = 10.0  # The maximum time for the simulation
ALPHA = 1  # Dissipative constant in the LLG equation.
THETA = 0.75 # Should be strictly above 1/2 for unconditional stability
K = 0.1  # TIME STEP
KAPPA = 1  # Determines the relative strength of the elastic vs. magnetic parts.
H_MAX = 0.2  # Determines how fine the mesh should be.

In [2]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left  = Plane(Pnt(0,0,0), Vec(-1,0,0)).bc("left")
    right = Plane(Pnt(1,1,1), Vec( 1,0,0)).bc("right")
    front = Plane(Pnt(0,0,0), Vec(0,-1,0)).bc("front")
    back  = Plane(Pnt(1,1,1), Vec(0, 1,0)).bc("back")
    bot   = Plane(Pnt(0,0,0), Vec(0,0,-1)).bc("bot")
    top   = Plane(Pnt(1,1,1), Vec(0,0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add (cube)
    #cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    geometry.Add(cube)
    return geometry


ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [3]:
  # this helps me figure out which sides are which.
#fes_SCALAR = H1(mesh, order=1)
#myScalars = {"left":10, "right":10, "front":0, "back":0, "bot":5, "top":5}
#SCALAR_gfu = CoefficientFunction([myScalars[val] for val in mesh.GetBoundaries()])
#Draw(SCALAR_gfu, mesh, "SCALAR_gfu")


In [4]:
fes_mag = VectorH1(mesh, order=1)  # the finite element space for the magnetisation m_h^i
fes_disp = VectorH1(mesh, order=1, dirichlet="bot")  # the finite element space for the displacement u_h^i

print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)
# body force and traction force

#f_body = CoefficientFunction((0,0,-1))
#g_surface = CoefficientFunction([-1 if bc=="left" else 0 for bc in mesh.GetBoundaries()])

mag_ndof=669, disp_ndof=669,
, dispfree_ndof=0: 01001101111111110000111111111111111111110000000000
50: 00111111111111111111111111111111111111111111111111
100: 11111111111111111111111000000000000000000011111111
150: 11111111111111111111111111111111111111111111111111
200: 11111111111111111111111010011011111111100001111111
250: 11111111111110000000000001111111111111111111111111
300: 11111111111111111111111111111111111111111111110000
350: 00000000000000011111111111111111111111111111111111
400: 11111111111111111111111111111111111111111111110100
450: 11011111111100001111111111111111111100000000000011
500: 11111111111111111111111111111111111111111111111111
550: 11111111111111111110000000000000000000111111111111
600: 11111111111111111111111111111111111111111111111111
650: 1111111111111111111


In [5]:
# Initial conditions
mag_gfu = magfunc.give_random_magnetisation(mag_gfu)

disp_gfu = elfunc.give_random_displacement(disp_gfu)
velocity_gfu = elfunc.give_random_displacement(disp_gfu)  # An initial velocity. Should only be used once in iteration.

In [6]:
# Test functions
v = fes_mag.TrialFunction()
phi = fes_mag.TestFunction()
# Building the linear system for the magnetisation

a_mag = BilinearForm(fes_mag)
a_mag += ALPHA*InnerProduct(v, phi)*dx  # α<v,Φ>
a_mag += THETA*K*InnerProduct(Grad(v), Grad(phi))*dx  # θk<∇v,∇Φ>
a_mag += InnerProduct(Cross(mag_gfu,v), phi)*dx  # <m×v,Φ>
a_mag.Assemble()

f_mag = LinearForm(fes_mag)
f_mag += -InnerProduct(Grad(mag_gfu), Grad(phi))*dx # -<∇m,∇Φ>
#f_mag +=  # magnetostrictive contribution.
f_mag.Assemble()

In [7]:
# Test functions
u = fes_disp.TrialFunction()
psi = fes_disp.TestFunction()
# Building the linear system for the displacement
a_disp = BilinearForm(fes_disp)
a_disp += 1/(K*K)*InnerProduct(u, psi)*dx
a_disp += InnerProduct(elfunc.stress(elfunc.strain(u)), elfunc.strain(psi))*dx

f_disp = LinearForm(fes_disp)
#f_mag += InnerProduct(elfunc.stress(elfunc.strain_m()), ψ)*dx  # <Cε_m(Π m),ψ>
#f_mag += 1/K*InnerProduct(diff_t u, psi)*dx  # <d_t u^i, ψ>
f_disp += 1/(K*K)*InnerProduct(disp_gfu, psi)*dx  # <u^i, ψ>
#f_mag += InnerProduct(f_body, psi)*dx  # <f, ψ>
#f_mag += InnerProduct(g_surface, psi)*ds  # _/‾ g·ψ ds

In [8]:
#B_T = magfunc.build_tangent_plane_matrix_transpose(mag_gfu)
with np.printoptions(threshold=np.inf):
    mag_gfux, mag_gfuy, mag_gfuz = mag_gfu.components
    print(type(magfunc.build_tangent_plane_matrix(mag_gfu)))

<class 'numpy.ndarray'>


In [9]:
print(mag_gfu.vec)

 -0.25335
 0.10113
 0.264345
 -0.758332
 0.482893
 -0.345536
 -0.0805983
 -0.369161
 0.680346
 0.607916
 0.647405
 -0.386899
 0.724434
 0.322265
 0.599034
 -0.901451
 0.202118
 -0.943916
 -0.802423
 0.339271
 -0.171623
 0.450018
 0.791656
 -0.835966
 -0.367459
 -0.460199
 -0.861333
 -0.324723
 -0.361432
 -0.181022
 -0.268168
 -0.34242
 0.202039
 0.36847
 -0.937973
 0.0187939
 -0.744279
 0.344468
 0.227034
 -0.422416
 0.594523
 0.488268
 0.642634
 -0.834235
 -0.503052
 -0.00501678
 -0.737833
 0.455958
 0.824462
 0.840682
 0.405374
 0.58366
 -0.327168
 -0.721674
 -0.619339
 0.448561
 -0.128019
 0.576632
 0.57334
 -0.477773
 -0.178045
 -0.611892
 0.273206
 0.318751
 -0.315311
 -0.0760575
 -0.52392
 -0.378081
 0.769303
 -0.0793046
 0.875333
 -0.79161
 0.718139
 -0.166746
 -0.647512
 0.205686
 -0.192652
 -0.753156
 0.602242
 0.360744
 0.548258
 0.310397
 -0.647329
 -0.585374
 -0.726082
 0.188084
 0.296637
 0.32653
 -0.598154
 -0.731833
 0.817634
 0.550135
 0.744865
 0.609271
 -0.425272
 -0.

In [11]:
#print(mag_gfu.vec)
Draw(mag_gfu)
print(disp_gfu.vec)


 -0.172679
 0.373065
 -0.964963
 -0.849729
 -0.359868
 -0.692358
 -0.623163
 -0.704428
 -0.61421
 -0.0940815
 0.653874
 0.90716
 -0.964946
 -0.552121
 -0.8115
 0.797826
 -0.902169
 0.978841
 0.147336
 -0.608531
 0.150495
 -0.543065
 0.0697315
 0.567273
 0.449153
 -0.0372693
 -0.75645
 0.0788358
 0.245964
 0.449422
 -0.844737
 0.532125
 0.580397
 -0.895803
 -0.523488
 -0.621391
 0.0986894
 -0.617705
 -0.149581
 0.910338
 0.208095
 -0.199181
 0.406878
 0.949364
 -0.684636
 -0.32866
 -0.566477
 -0.0347335
 0.998374
 0.208756
 -0.93801
 0.974562
 -0.828471
 -0.725126
 -0.761041
 0.937969
 0.508046
 -0.00685281
 0.565387
 0.95187
 0.575185
 0.838659
 0.422072
 0.882745
 0.944021
 -0.129518
 -0.633027
 0.441385
 0.0688245
 -0.818901
 -0.575259
 -0.0603344
 0.483193
 -0.630682
 0.24234
 -0.177219
 0.894791
 0.784982
 0.364074
 -0.475889
 -0.639314
 0.152829
 -0.554447
 0.573532
 0.451146
 0.943105
 0.112774
 -0.0653747
 -0.589746
 0.799601
 -0.458344
 -0.377918
 -0.924037
 -0.0626615
 0.57821